https://projecteuler.net/problem=secret

In [75]:
import cv2
import numpy as np
from tqdm import tqdm
import numpy as np

In [ ]:
def transform_fft(image, n_iter):
    image = image % 7
    N, M = image.shape

    kernel = np.zeros((N, M), dtype=np.int32)
    kernel[0, 1] = 1
    kernel[0, -1] = 1
    kernel[1, 0] = 1
    kernel[-1, 0] = 1

    F_image = np.fft.fft2(image)
    F_kernel = np.fft.fft2(kernel)

    F_result = F_image * (F_kernel ** n_iter)
    result = np.fft.ifft2(F_result).real
    result = np.round(result) % 7
    return result.astype(np.uint8)

def arr_to_img(arr):
    arr = np.clip(arr, 0, 255).astype(np.uint8)
    cv2.imwrite("output.jpg", arr)

In [ ]:
def arr_to_img(arr):
    arr = np.clip(arr, 0, 255).astype(np.uint8)
    cv2.imwrite("output.jpg", arr)

In [84]:
import numpy as np
import cv2

def circular_convolve_mod7(image):
    N, M = image.shape
    top    = np.roll(image,  1, axis=0)
    bottom = np.roll(image, -1, axis=0)
    left   = np.roll(image,  1, axis=1)
    right  = np.roll(image, -1, axis=1)
    return (top + bottom + left + right) % 7

def transform_exact(image, n_iter):
    result = image % 7
    power = circular_convolve_mod7
    # Use repeated squaring idea
    # For large n, we cannot literally iterate; we implement a matrix-like doubling trick
    def mat_pow(arr, exp):
        if exp == 1:
            return circular_convolve_mod7(arr)
        if exp % 2 == 0:
            half = mat_pow(arr, exp // 2)
            return circular_convolve_mod7(half)  # Convolution squared (mod 7)
        else:
            half = mat_pow(arr, exp - 1)
            return circular_convolve_mod7(half)
    return mat_pow(result, n_iter)

def arr_to_img(arr):
    arr = np.clip(arr * (255 // 6), 0, 255).astype(np.uint8)
    cv2.imwrite("output.jpg", arr)

img = cv2.imread("secret.jpg", cv2.IMREAD_GRAYSCALE).astype(np.int32)
img_final = transform_exact(img, 10**12)
arr_to_img(img_final%7)


In [82]:
img %7

array([[1, 3, 2, ..., 0, 0, 6],
       [3, 6, 5, ..., 1, 0, 6],
       [1, 5, 6, ..., 1, 1, 0],
       ...,
       [3, 2, 3, ..., 1, 5, 2],
       [5, 5, 6, ..., 3, 1, 5],
       [5, 2, 0, ..., 3, 5, 1]], shape=(585, 975), dtype=int32)